In [ ]:
!pip3 install ktrain

In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
default_dir = "/data/"
data_tcc_pos_neg = default_dir+'labeled_data/dataset_label_pos_neg.csv'
data_neg_emotions = default_dir+'labeled_data/dataset_neg_emotions.csv'


In [ ]:
data_base = pd.read_csv(data_tcc_pos_neg)
data['type'] = 'with_theme-' + data['emotion']
data.shape

In [ ]:
plot = data['emotion'].value_counts().plot.pie(y='mass', figsize=(10, 15))

In [ ]:
from sklearn.model_selection import train_test_split

def create_splits(data):
    test_validation_size = int(0.30*data.shape[0])
    train, test = train_test_split(data, test_size=test_validation_size, random_state=42, stratify=data['type'])
    return train, test

  
train, test = create_splits(data)
print('Training samples:  ', train.shape[0])
print('Test samples:      ', test.shape[0])


In [ ]:
X_train = train.cleaned_text.tolist()
X_test = test.cleaned_text.tolist()

y_train = train.emotion.tolist()
y_test = test.emotion.tolist()

In [ ]:
# encoding = {
#     'Positivo' : 0,
#     'Negativo' : 1,
# }

encoding = {
    'tristeza': 0,
    'medo': 1,
    'raiva': 2,
    'desprezo' : 3,
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [ ]:
class_names = list(encoding.keys())

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=350000)

preprocessing train...
language: pt


Is Multi-Label? False
preprocessing test...
language: pt


task: text classification


In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)


Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)
learner.fit_onecycle(2e-5, 3)




begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
7949/7949 [==============================] - 5548s 695ms/step - loss: 0.6081 - accuracy: 0.7140 - val_loss: 0.5967 - val_accuracy: 0.7299
Epoch 2/3
7949/7949 [==============================] - 5422s 682ms/step - loss: 0.5589 - accuracy: 0.7417 - val_loss: 0.5517 - val_accuracy: 0.7520
Epoch 3/3
7949/7949 [==============================] - 5426s 683ms/step - loss: 0.5121 - accuracy: 0.7725 - val_loss: 0.5318 - val_accuracy: 0.7613


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)


              precision    recall  f1-score   support

    tristeza       0.45      0.58      0.51      2114
        medo       0.93      0.78      0.85      3480
       raiva       0.83      0.80      0.81      4948
    desprezo       0.77      0.79      0.78      5354

    accuracy                           0.76     15896
   macro avg       0.74      0.74      0.74     15896
weighted avg       0.78      0.76      0.77     15896



array([[1229,   62,  318,  505],
       [ 353, 2719,  141,  267],
       [ 458,   58, 3949,  483],
       [ 684,   88,  377, 4205]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()